In [1]:
import pandas as pd
import os
import numpy as np
import keras as K
import pydicom
import cv2
from pathlib import Path
import matplotlib
import pickle
# matplotlib.use('Agg')
from matplotlib import pyplot as plt
from PIL import Image

Using TensorFlow backend.


**READ IN PATIENT INFO**

In [2]:
#Read 
patient_info = pd.read_csv(Path('Methylation/patient_info.csv').absolute())

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,9,10,11,23,25,27,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
patient_info = patient_info[['Patient ID','modality_path','GivenModalityForThisStudy','ModalityName']].groupby(['Patient ID','modality_path','GivenModalityForThisStudy']).count().reset_index()
patient_info = patient_info.rename(columns={'ModalityName': 'count'})
patient_info = patient_info.drop_duplicates('modality_path')

In [4]:
len(patient_info.loc[patient_info['GivenModalityForThisStudy']=='t1'])

131

In [5]:
from custom_metrics import dice_coef, soft_dice_coef, dice_coef_loss, combined_dice_ce_loss

In [6]:
custom_objects = {
"combined_dice_ce_loss": combined_dice_ce_loss,
"dice_coef_loss": dice_coef_loss,
"dice_coef": dice_coef,
"soft_dice_coef": soft_dice_coef}

**LOAD TRAINED BRATS UNET MODEL**

In [7]:
import keras as K
unet_model = K.models.load_model(str(Path('unet/2D/output/unet_model_for_decathlon.hdf5').absolute()), custom_objects=custom_objects)

In [8]:
input_channels = {
     "flair": 0,
     "t1": 1,
     "t1Gd": 2,
     "t2": 3
 }

In [9]:
#Subset T1
patient_info_t1 = patient_info.loc[patient_info['GivenModalityForThisStudy']=='t1']

In [10]:
patient_info.head()

,Patient ID,modality_path,GivenModalityForThisStudy,count
0,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t2,25
1,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,flair,25
2,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t1,25
3,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t1Gd,25
4,TCGA-02-0006,/external_drive/MRI_methylation/Methylation/da...,t2,23


**USE TRAINED UNET TO PREDICT ON TCGA DATA**

In [11]:
frames_with_tumour = []
save_slices = True

for idx, row in patient_info_t1.iterrows():
    MRI_vid = []
    patient_frames_w_tumour = []
    for dicom in os.listdir(row['modality_path']):
        dcm = pydicom.dcmread(os.path.join(row['modality_path'],dicom))
        print(os.path.join(row['modality_path'],dicom))
        img = dcm.pixel_array
        try:
            img = dcm.pixel_array*dcm.RescaleSlope+dcm.RescaleIntercept
        except:
            pass
        try:
            min_value, max_value = dcm.WindowCenter - dcm.WindowWidth/2, dcm.WindowCenter + dcm.WindowWidth/2
            img[img < min_value] = min_value
            img[img > max_value] = max_value
        except:
            pass
#         img = cv2.resize(img, (144,144))
        if (np.mean(img)<20) & (np.std(img)<20):
            pass
        else:
            original_img = img.astype(np.float32)
            img = (img - np.mean(img))/np.std(img)
            img = img.reshape(img.shape+(1,))
            pred = unet_model.predict(np.array([img]))[0,:,:,0]
            tumour_size = len(np.where(pred.flatten()>0.5)[0])
            #Save slices to disk
            if save_slices==True:
                if tumour_size>(0.01*dcm.pixel_array.shape[0]*dcm.pixel_array.shape[1]):
                    ret, labels = cv2.connectedComponents(pred.astype(np.uint8))
                    (values,counts) = np.unique(labels,return_counts=True)
                    values = values[1:]
                    counts = counts[1:]
                    ind=np.argmax(counts)
                    mask = np.where(labels==values[ind],1,0)
                    x, y, width, height = cv2.boundingRect(mask.astype(np.uint8))
                    roi = original_img[y:y+height, x:x+width]
                    if roi.all()!=0:
                        os.makedirs(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour_slices'),exist_ok=True)                                          
                        with open(os.path.join(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour_slices'),
                            dicom.replace('dcm','pickle')),'wb') as p:
                            pickle.dump(roi,p) 
                        roi = cv2.resize(roi,(128,128))
                        im = Image.fromarray(roi.astype(np.uint8))
                        im.save(os.path.join(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour_slices'),
                                             dicom.replace('dcm','jpg')))                  
#             #Save images w masks to disk
#             if tumour_size>0:
#                 patient_frames_w_tumour.append(dicom)
#                 mask = np.where(pred>0.5,np.max(original_img.flatten()),0.0).astype(np.float32)
#                 plt.figure(figsize=(10, 6.67))
#                 plt.subplot(1, 2, 1)
#                 plt.imshow(original_img, cmap="bone", origin="lower")
#                 plt.title("MRI")
#                 plt.axis("off")
#                 plt.subplot(1, 2, 2)
#                 plt.imshow(mask, origin="lower")
#                 plt.title("Prediction")
#                 plt.axis("off")
#                 os.makedirs(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour-mask'),exist_ok=True)
#                 plt.savefig(os.path.join(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour-mask'),
#                                      dicom.replace('dcm','jpg')), bbox_inches='tight')                    
# #                 overlayed_mask = cv2.addWeighted(original_img,1.0,mask,0.5,0.0).astype(np.uint8)
# #                 im = Image.fromarray(overlayed_mask)
# #                 im.save(os.path.join(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour-mask'),
# #                                      dicom.replace('dcm','jpg')))             
#     frames_with_tumour.append(sorted(patient_frames_w_tumour))

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0003/06-08-1997-MRI BRAIN WWO CONTRAMR-81239/12-AX T1-90444/000015.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0003/06-08-1997-MRI BRAIN WWO CONTRAMR-81239/12-AX T1-90444/000017.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0003/06-08-1997-MRI BRAIN WWO CONTRAMR-81239/12-AX T1-90444/000024.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0003/06-08-1997-MRI BRAIN WWO CONTRAMR-81239/12-AX T1-90444/000009.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0003/06-08-1997-MRI BRAIN WWO CONTRAMR-81239/12-AX T1-90444/000005.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0003/06-08-1997-MRI BRAIN WWO CONTRAMR-81239/12-AX T1-90444/000007.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0003/06-08-1997-MRI BRAIN WWO CONTRAMR-81239/12-AX T1-90444/000018.dcm
/external_drive/MRI_methylation/Methylati

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0009/06-14-1997-MRI BRAIN WWO CONTRAMR-13452/5-AX T1-38458/000021.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0009/06-14-1997-MRI BRAIN WWO CONTRAMR-13452/5-AX T1-38458/000008.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0009/06-14-1997-MRI BRAIN WWO CONTRAMR-13452/5-AX T1-38458/000019.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0009/06-14-1997-MRI BRAIN WWO CONTRAMR-13452/5-AX T1-38458/000014.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0009/06-14-1997-MRI BRAIN WWO CONTRAMR-13452/5-AX T1-38458/000022.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0009/06-14-1997-MRI BRAIN WWO CONTRAMR-13452/5-AX T1-38458/000000.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0009/06-14-1997-MRI BRAIN WWO CONTRAMR-13452/5-AX T1-38458/000012.dcm
/external_drive/MRI_methylation/Methylation/data

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/12-AX T1-46175/000023.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/12-AX T1-46175/000010.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/12-AX T1-46175/000001.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/12-AX T1-46175/000020.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/12-AX T1-46175/000016.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/12-AX T1-46175/000002.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/12-AX T1-46175/000011.dcm
/external_drive/MRI_methylation/Methylati

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pydicom/filereader.py:313: UserWarning: Expected explicit VR, but found implicit VR - using implicit VR for reading
  warnings.warn(message, UserWarning)


/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0046/11-28-1998-MRI BRAIN WWO CONTR-34177/5-AX T1-73320/000003.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0046/11-28-1998-MRI BRAIN WWO CONTR-34177/5-AX T1-73320/000004.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0046/11-28-1998-MRI BRAIN WWO CONTR-34177/5-AX T1-73320/000010.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0046/11-28-1998-MRI BRAIN WWO CONTR-34177/5-AX T1-73320/000001.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0046/11-28-1998-MRI BRAIN WWO CONTR-34177/5-AX T1-73320/000020.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0046/11-28-1998-MRI BRAIN WWO CONTR-34177/5-AX T1-73320/000016.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-0046/11-28-1998-MRI BRAIN WWO CONTR-34177/5-AX T1-73320/000002.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-02-004

...

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-4935/01-22-2001-MRI BRAIN COMBO-10861/1101-T1 AX  SE-88216/000006.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6282/08-24-1998-JHN BRAIN COMBO IP-22866/4-BRAINSTRYKERT1AXIAL-28304/000015.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6282/08-24-1998-JHN BRAIN COMBO IP-22866/4-BRAINSTRYKERT1AXIAL-28304/000017.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6282/08-24-1998-JHN BRAIN COMBO IP-22866/4-BRAINSTRYKERT1AXIAL-28304/000009.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6282/08-24-1998-JHN BRAIN COMBO IP-22866/4-BRAINSTRYKERT1AXIAL-28304/000005.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6282/08-24-1998-JHN BRAIN COMBO IP-22866/4-BRAINSTRYKERT1AXIAL-28304/000007.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6282/08-24-1998-JHN BRAIN COMBO IP-22866/4-BRAINSTRYKERT1AXIAL-28304

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02-2001-MRI BRAIN IP-02901/901-T1 AX  SE-25444/000002.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02-2001-MRI BRAIN IP-02901/901-T1 AX  SE-25444/000026.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02-2001-MRI BRAIN IP-02901/901-T1 AX  SE-25444/000011.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02-2001-MRI BRAIN IP-02901/901-T1 AX  SE-25444/000021.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02-2001-MRI BRAIN IP-02901/901-T1 AX  SE-25444/000008.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02-2001-MRI BRAIN IP-02901/901-T1 AX  SE-25444/000028.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02-2001-MRI BRAIN IP-02901/901-T1 AX  SE-25444/000019.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6656/06-02

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6661/12-08-2001-MRIA  BRAIN CONTRAST JHN-34840/1101-T1 AX  SE-45423/000005.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6661/12-08-2001-MRIA  BRAIN CONTRAST JHN-34840/1101-T1 AX  SE-45423/000007.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6661/12-08-2001-MRIA  BRAIN CONTRAST JHN-34840/1101-T1 AX  SE-45423/000018.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6661/12-08-2001-MRIA  BRAIN CONTRAST JHN-34840/1101-T1 AX  SE-45423/000013.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6661/12-08-2001-MRIA  BRAIN CONTRAST JHN-34840/1101-T1 AX  SE-45423/000003.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6661/12-08-2001-MRIA  BRAIN CONTRAST JHN-34840/1101-T1 AX  SE-45423/000004.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6661/12-08-2001-MRIA  BRAIN CONTRAST JHN-34840/1101-T1 AX  SE-45

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6662/12-13-2001-MRI BRAIN WITH-86043/1001-T1 AX SE-51871/000011.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6662/12-13-2001-MRI BRAIN WITH-86043/1001-T1 AX SE-51871/000021.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6662/12-13-2001-MRI BRAIN WITH-86043/1001-T1 AX SE-51871/000008.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6662/12-13-2001-MRI BRAIN WITH-86043/1001-T1 AX SE-51871/000019.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6662/12-13-2001-MRI BRAIN WITH-86043/1001-T1 AX SE-51871/000014.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6662/12-13-2001-MRI BRAIN WITH-86043/1001-T1 AX SE-51871/000022.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6662/12-13-2001-MRI BRAIN WITH-86043/1001-T1 AX SE-51871/000000.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA

/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6663/12-27-2001-MRI BRAIN WITH STRYKER-65499/801-T1 AX  SE-74602/000018.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6663/12-27-2001-MRI BRAIN WITH STRYKER-65499/801-T1 AX  SE-74602/000013.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6663/12-27-2001-MRI BRAIN WITH STRYKER-65499/801-T1 AX  SE-74602/000003.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6663/12-27-2001-MRI BRAIN WITH STRYKER-65499/801-T1 AX  SE-74602/000004.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6663/12-27-2001-MRI BRAIN WITH STRYKER-65499/801-T1 AX  SE-74602/000010.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6663/12-27-2001-MRI BRAIN WITH STRYKER-65499/801-T1 AX  SE-74602/000001.dcm
/external_drive/MRI_methylation/Methylation/data/TCGA-GBM/TCGA-76-6663/12-27-2001-MRI BRAIN WITH STRYKER-65499/801-T1 AX  SE-74602/000020.dcm
/exter

In [ ]:
patient_info_t1['frames_with_tumour'] = frames_with_tumour

In [ ]:
patient_info_t1.head()

**SUBSET PATIENT INFO TO INCLUDE ONLY T1 PATIENTS AND SAVE TO DISK**

In [ ]:
patient_info_t1.to_pickle(str(Path('Methylation/patient_info_t1.pickle').absolute()))